In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 加载模型
model_path = "models/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    use_safetensors=True,
    num_labels=2
)

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return "正面" if predicted_class == 1 else "负面"

# 测试数据
movie_review = "剧情设定新颖不落俗套，每个转折都让人惊喜。"
food_review = "这家店口味稳定，已经回购好几次了，值得信赖！"

# 执行预测
print(f" {movie_review} → {predict_sentiment(movie_review)}")
print(f" {food_review} → {predict_sentiment(food_review)}")




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 剧情设定新颖不落俗套，每个转折都让人惊喜。 → 正面
 这家店口味稳定，已经回购好几次了，值得信赖！ → 正面


In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_path = "models/gpt2-chinese-cluecorpussmall"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

def generate_text(prompt, max_length=100):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
        return_attention_mask=True
    ).to(device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        min_length=len(inputs.input_ids[0]) + 20,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # 解码输出
    generated_text = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )
    return generated_text


# 测试生成
prompt = "当人类第一次踏上火星"
print("生成结果:", generate_text(prompt))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


生成结果: 当 人 类 第 一 次 踏 上 火 星 当 宇 航 员 遇 上 宇 宙 大 战 。 。 对 于 女 性 来 说 人 体 的 内 在 机 能 必 须 和 人 的 生 理 结 构 有 很 大 的 关 系 。 当 时 美 国 发 现 了 可 以 改 善 人 们 的 体 能 的 新 材 料 ， 那 就 是 人 造 的 血 管 血 流 管 。 血 液 管 中 是 血 生 成 的 。 而
